In [ ]:
pip install requests pandas matplotlib unidecode nltk numpy==1.19.3 spacy==2.2.4


In [ ]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import re
import unidecode
import string
import spacy
import nltk
from nltk.tokenize import word_tokenize
sp = spacy.load('en_core_web_sm')
nltk.download('punkt')

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def createPie(data: dict):
    saliences = [x*100 for x in data['salience']]
    wedges = plt.pie(
        saliences, labels=data['types'], shadow=True, autopct='%1.1f%%', startangle=90)
    plt.show()


def createTable(data: list):
    types = []
    salience = []
    entities = []
    count = []
    for x in data:
        types.append(x['type'])
        salience.append(x['salience'])
        entities.append(x['entities'])
        count.append(len(x['entities']))
        print('Entities for Type '+x['type']+' is ')
        print(x['entities'])
    stemdf = pd.DataFrame({'type': types, 'salience': salience,
                           'entitiesCount': count, 'entities': entities})
    print('\n')
    print('\033[1m'+'Tabular format'+'\033[0m'+'\n')
    print(stemdf)
    return {"salience": salience, "types": types}
def filterStopWords(data: string):
    all_stopwords = sp.Defaults.stop_words
    # all_stopwords.remove("not")
    text_tokens = word_tokenize(data)
    tokens_without_sw = [
        word for word in text_tokens if not word in all_stopwords]
    return " ".join([token for token in tokens_without_sw])


def lemmatization(data: string):
    doc = sp(data)
    return " ".join([token.lemma_ for token in doc])


def filterSpecialChar(data: list):
    alphaNumericData = []
    for x in data:
        alphaNumericData.append(x.translate(
            str.maketrans('', '', string.punctuation)))
    return alphaNumericData


def preprocessData(data: list):
    data = [''.join(x) for x in data]  # convert to list of string
    data = list(dict.fromkeys(data))  # remove duplicate values
    data = list(filter(None, data))   # remove empty values
    data = filterSpecialChar(data)  # remove special characters
    data = [re.sub("(<.*?>)", "", word) for word in data]  # remove html markup
    # Convert accented characters to ASCII characters
    data = [unidecode.unidecode(word) for word in data]
    data = [word.strip() for word in data]  # remove trailing space
    data = [decontracted(word)
            for word in data]  # handled contractions
    data = [filterStopWords(word) for word in data]  # filter StopWords
    data = [lemmatization(word) for word in data]  # Lemmatization
    data = appendIfAbsent('.', data)        # add . in end of line
    data = [word.lower() for word in data]  # lower case
    data = [word.replace('-pron-', '') for word in data]
    data = [word.strip() for word in data]  # remove trailing space

    return data


def appendIfAbsent(s, data):
    fixedData = []
    for x in data:
        fixedData.append(
            x.lower() + s) if not x.endswith(s) else fixedData.append(x.lower())
    return fixedData


In [ ]:
import requests
import json

data = ["The vet was very good", " My cat did not like the vet, but my dog did", " I do not like vets, they are too expensive", "I love my vet", " I love my dog", " I like brown cats best", " I like brown dogs", " The cat jump through a window and injured itself", "I enjoyed the experience", " I would recommend my vet", " The vet was good, but I had to wait too long and my cat", " I have farm animals and the vet does not support them", " I like horses", " My cat is my best friend and I give him the best", "I valued the transaction", " I thought the service was over priced", " I do not think the vet likes cats", " My cat does not like vets", " I had to wait to long", " It was very expensive", " I was not told how much it would cost", "I have three cats and a rabbit and take them all to the same vet",
        " My cat was very sick and the vet was fantastic", "I wish the vet was quicker", " I had to wait to long and would not recommend this vet", "I wanted to by insurance for my pet, but they would not", " My dog misbehaved at the vet", " I adopted my dog from this vet", " I need a microchip scanned", " The vet was very sensitive to my needs at a challenging time", " The vet lost my cat and I was devastated", " I had a budgie, but it escaped at the vet", " The medicine was very expensive", " I need to take the cat back to the vet many times", " I liked the vet", " I miss my original vet", " I wish my doctor was as good as my vet", " Why is the vet more expensive than my doctor? I am happy with the treatment given by my vet", " My cat caught fleas at the vet", " She needed a vaccination"]

url = 'https://text-classification-engine-rest-dot-feijoa8-dev.ts.r.appspot.com/api/v1/text/classify/entity-sentiments'
header = {'Content-Type': 'application/json',
          'industryVertical': 'testing', 'Accept': 'application/json'}


requestBody = {"data": data}
response = requests.post(url, json=requestBody, headers=header)
response_dict = json.loads(response.text)

preProcesseddata = preprocessData(data)
requestBodyPreprocess = {"data": preProcesseddata}
responsePrepProcess = requests.post(
    url, headers=header, json=requestBodyPreprocess)
response_dict_preprocess = json.loads(responsePrepProcess.text)

print('\033[1m'+'Without data preprocessing'+'\033[0m'+'\n')
print('\n')
value = createTable(response_dict)
print('\n')
print('\033[1m'+'Pie'+'\033[0m'+'\n')
createPie(value)
print('\n')
print('\n')
print('\n')
print('\033[1m'+'With data preprocessing'+'\033[0m'+'\n')
print('\n')
preProcessedValue = createTable(response_dict_preprocess)
print('\n')
print('\033[1m'+'Pie'+'\033[0m'+'\n')
createPie(preProcessedValue)
